In [2]:
#Importation
import pickle
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sklearn as sk
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from statistics import mean
from math import ceil
from collections import Counter
warnings.filterwarnings('ignore')

In [3]:
# On ramene les données du prétraitement du dataframe
df= pd.read_pickle("../PICKLE/df_stat_commentaires.pkl")
df

,score_reussite,score_echec,nombre_commentaires,Longueur moyenne commentaire,Reussite,polarite,subjectivite
0,-1.365733,-0.869921,-0.755824,-0.328797,False,0.000000,0.000000
1,0.457547,0.164916,0.333062,-0.468411,True,0.100000,0.100000
2,1.567866,1.617878,1.966390,1.752941,True,0.062500,0.062500
3,-0.173371,-0.744961,-0.755824,-0.627969,True,0.133333,0.158333
4,-0.543328,-0.751616,-0.755824,-0.877279,True,0.166667,0.166667
...,...,...,...,...,...,...,...
4183,-0.191482,0.146861,1.966390,-0.359961,True,0.000000,0.000000
4184,0.065179,-0.596722,-0.755824,-0.665366,True,0.166667,0.166667
4185,-0.582355,-0.757317,-0.755824,-0.627969,True,0.166667,0.166667
4186,-0.305364,-0.483163,0.333062,-0.722707,True,0.100000,0.100000


In [4]:
def creation_test_train(dataframe,label):
    train_df = dataframe.sample(frac=0.8)
    test_df = dataframe.drop(train_df.index)

    train_df_Y = train_df[label]
    test_df_Y = test_df[label]

    train_df_X = train_df.drop(label,axis=1)
    test_df_X = test_df.drop(label,axis=1)
    
    return train_df_X,train_df_Y,test_df_X,test_df_Y

In [5]:
def stat_modele(true,pred,arrondi=5,affichage=True):
    accuracy = round(sk.metrics.accuracy_score(true,pred),arrondi)*100
    rappel = round(sk.metrics.recall_score(true,pred),arrondi)*100
    precision = round(sk.metrics.precision_score(true,pred),arrondi)*100
    specificity = round(sk.metrics.recall_score(true,pred,pos_label=0),arrondi)*100
    vpn = round(sk.metrics.precision_score(true,pred,pos_label=0),arrondi)*100
    if affichage :
        print("Accuracy : proportion de prédictions correctes (VP + VN /(VP+VN+FN+FP)) : " + str(accuracy) + "%")
        print("Rappel : proportion de réussites correctement prédites sur l'ensemble des réussites (VP/(VP+FN)) : " + str(rappel) + "%")
        print("Précision :  proportion de véritables réussites parmi les réussites prédites (VP/(VP+FP)) : " + str(precision) + "%")
        print("Spécificité : proportion d'échecs correctement prédits sur l'ensemble des échecs (VN/(VN+FP)) : "+str(specificity) + "%")
        print("Valeur prédictive négative: proportion de véritables échecs parmi les échecs prédits (VN/(VN+FN)) : " + str(vpn) + "%")
   
    return accuracy,rappel,precision,specificity,vpn

In [6]:
def arbre():
    train_df_X,train_df_Y,test_df_X,test_df_Y = creation_test_train(df,"Reussite")
    arbre = RandomForestClassifier(criterion="gini", min_samples_leaf=2,max_depth=6, random_state=42)
    arbre.fit(train_df_X, train_df_Y)
    
    return arbre

### Premier modèle

In [7]:
arbre()

RandomForestClassifier(max_depth=6, min_samples_leaf=2, random_state=42)

### Modèle avec seuil

In [8]:
def arbre_seuil():
    train_df_X,train_df_Y,test_df_X,test_df_Y = creation_test_train(df,"Reussite")
    arbre = RandomForestClassifier(criterion="gini", min_samples_leaf=2,max_depth=6, random_state=42)
    arbre.fit(train_df_X, train_df_Y)
    predictions = arbre.predict_proba(test_df_X)
    seuil = 0.8
    predict_label_seuil = [False if proba[0]>=seuil else True for proba in predictions]
    stat_modele(test_df_Y,predict_label_seuil,arrondi=5,affichage=True)
    return arbre

In [13]:
arbre_seuil()

Accuracy : proportion de prédictions correctes (VP + VN /(VP+VN+FN+FP)) : 90.095%
Rappel : proportion de réussites correctement prédites sur l'ensemble des réussites (VP/(VP+FN)) : 99.864%
Précision :  proportion de véritables réussites parmi les réussites prédites (VP/(VP+FP)) : 89.96300000000001%
Spécificité : proportion d'échecs correctement prédits sur l'ensemble des échecs (VN/(VN+FP)) : 19.608%
Valeur prédictive négative: proportion de véritables échecs parmi les échecs prédits (VN/(VN+FN)) : 95.238%


RandomForestClassifier(max_depth=6, min_samples_leaf=2, random_state=42)

### Finetune

In [51]:
def BestHyperparameters_vpn(trainX,trainY,testX,testY,min_example,depth):
    best_vpn = 0
    best_min_example = 0
    best_depth = 0
    best_criterion = ""
    #On va tester toutes les combinaisons possibles :
    for criterion in ["gini","entropy"]:
        for example in min_example:
            for profondeur in depth:
                #On créer l'arbre
                arbre = RandomForestClassifier(criterion=criterion,min_samples_leaf=example,max_depth=profondeur)
                arbre.fit(trainX, trainY)
                #On calcule la prédiction et la vpn associée
                predictionY = arbre.predict(testX)
                vpn = round(sk.metrics.precision_score(testY,predictionY,pos_label=0),5)*100
                #On compare
                if vpn>best_vpn :
                    best_vpn = vpn
                    best_min_example = example
                    best_depth = profondeur
                    best_criterion= criterion
                
    return (best_vpn,best_min_example,best_depth,best_criterion)

In [72]:
#Fonction permettant de trouver les meilleurs hyperparametres en refaisant les dataset d'entrainement de test plusieurs fois et en gardant la moyenne
def Finetune_vpn(nombre_essai,min_example,depth):
    l_vpn = []
    l_best_min_example = []
    l_best_depth = []
    l_best_criterion = []
    
    for i in range (0,nombre_essai) :
        print(i)
        trainX,trainY,testX,testY = creation_test_train(df,"Reussite")
        best_vpn,best_min_example,best_depth,best_criterion = BestHyperparameters_vpn(trainX,trainY,testX,testY,min_example,depth)
        l_vpn.append(best_vpn)
        l_best_min_example.append(best_min_example)
        l_best_depth.append(best_depth)
        l_best_criterion.append(best_criterion)
    return mean(l_vpn),Counter(l_best_min_example).most_common(1)[0][0],Counter(l_best_depth).most_common(1)[0][0],Counter(l_best_criterion).most_common(1)[0][0]
    

In [73]:
Finetune_vpn(50,[2,3],[2,3,4,5,6])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


(82.4483, 2, 6, 'gini')